# HyperPerimeter Tuning for the above model...
## pip install keras-tuner

In [1]:
from operator import imod
from keras.src.models import Sequential 
from keras.src.layers import Dense, Dropout, Input
from keras_tuner import RandomSearch 
from keras.src.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from keras.src.utils import to_categorical
import pandas as pd 

In [ ]:
## Spliting data into train and test data

df = pd.read_csv('../Data/Clean_Raw_Data')

X=df.iloc[:,:-1]
y=df['NObeyesdad']

x_train_full,  x_test, y_train_full , y_test =train_test_split(X,y, test_size=0.2, random_state=42)

x_train_hp, x_valid, y_train_hp, y_valid = train_test_split(x_train_full, y_train_full, random_state=42, test_size=0.2)


In [ ]:
## Scaling the data by using StandardScaler

hp_scaler = StandardScaler()

x_train_scaled_hp = hp_scaler.fit_transform(x_train_hp)
x_test_scaled_hp = hp_scaler.transform(x_valid)

In [ ]:
x_train_scaled_hp.shape

In [ ]:
## Converting  the target data into One-Hot vector

y_train_hp_cat = to_categorical(y_train_hp)
y_valid_cat = to_categorical(y_valid, 7)

y_train_hp.shape

In [ ]:
## Define the model-building function 

def build_model(hp):
    
    model = Sequential()
    
    # Input layer with tunable number of neurone
    model.add(Input(shape=(x_train_scaled_hp.shape[1],)))
    model.add(Dense(units=hp.Int('units_input', min_value=32, max_value=256, step=32),
                    activation='relu'))
    
    # first hidden layer with tunable units and dropout
    model.add(Dense(units=hp.Int('units_hidden1', min_value=64, max_value=256, step=32), activation='relu'))
    model.add(Dropout(rate=hp.Float('dropout_hidden1', min_value=0.2, max_value=0.5, step=0.1)))
    
    # second hidden layer
    model.add(Dense(units=hp.Int('unit_hidden2', min_value=32, max_value=128, step=32), activation='relu'))
    model.add(Dropout(rate=hp.Float('unit_dropout2', min_value=0.2, max_value=0.5, step=0.1)))
    
    # third hidden layer 
    model.add(Dense(units=hp.Int('unit_hidden3', min_value=16, max_value=64, step=32), activation='relu'))
    model.add(Dropout(rate=hp.Float('unit_dropout3', min_value=0.2, max_value=0.5, step=0.1)))
    
    # Output layer
    model.add(Dense(y_train_hp.shape[1], activation='softmax'))
    
    # Compie the model with a tunable learning rate
    model.compile(optimizer=Adam( learning_rate=hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4, 1e-5])),
                  loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [ ]:
## Initialize the tuner 

tuner = RandomSearch(build_model,
                     objective='val_accuracy',
                     max_trials=5,
                     executions_per_trial=3,
                     directory='hyperparameter_tuning',
                     project_name='Obesity Classification')

In [ ]:
## Start the search using your training data and validation data.

tuner.search(x_train_scaled_hp, y_train_hp_cat, epochs=10, validation_data=(x_valid, y_valid_cat))